In [2]:
import pandas as pd
import re
import requests
import os

from dotenv import load_dotenv

#.env 파일 로드
load_dotenv()
kakao_api_key = os.getenv("kakao_api_key")


### 청약매물 목록 위도, 경도, 행정구역 컬럼 추가

In [3]:
def pre_col_gen(df):

    # 중복된 '공고번호' 제거 (첫 번째 항목만 남김)
    df_copy = df.reset_index(drop=True).drop_duplicates(subset="공고번호", keep='first', ignore_index=True)
    
    # '정제_공급위치' 열 생성 후 간단한 전처리
    df_copy["정제_공급위치"] = df_copy["공급위치"].apply(lambda address: re.sub(r'\([^)]*\)', '', address).replace('블록', '').replace('일원', '').replace('공동', '').replace('일반', '').replace('  ', ' ').strip())
    
    # 필요한 열만 선택
    df_copy = df_copy[["공고번호", "정제_공급위치"]]
    
    return df_copy

def process_address_data(df):
    # 카카오맵 API 사용해서 위도 경도 추출
    kakao_api_url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {kakao_api_key}"}

    def get_lat_lon_kakao(address):
        try:
            response = requests.get(kakao_api_url, headers=headers, params={"query": address})
            if response.status_code == 200:
                result = response.json()
                if result['documents']:
                    lat = result['documents'][0]['y']
                    lon = result['documents'][0]['x']
                    return lat, lon
            return None, None
        except Exception as e:
            print(f"Error fetching data for address {address}: {e}")
            return None, None

    df['위도'], df['경도'] = zip(*df['정제_공급위치'].apply(lambda x: get_lat_lon_kakao(x)))

    return df

def process_address(df):
    
    def split_and_update_address(address):
        parts = address.split(' ')
        
        if len(parts) >= 3:
            sido = parts[0]
            sigungu = parts[1]
            eupmyeondong = parts[2]
            updated_address = f"{sido} {sigungu} {eupmyeondong}"
            return sido, sigungu, eupmyeondong, updated_address
        else:
            return None, None, None, address
    
    # 주소 분리 및 업데이트
    df[['시도', '시군구', '읍면동', '정제_공급위치_업데이트']] = df['정제_공급위치'].apply(lambda x: pd.Series(split_and_update_address(x)))
    
    # 기존 정제_공급위치 열을 업데이트함
    df['정제_공급위치'] = df['정제_공급위치_업데이트']
    
    # 불필요한 열 제거
    df = df.drop(columns=['정제_공급위치_업데이트'])
    
    return df


def lat_lon_pipeline(df):

    # 서울, 경기, 인천 지역만 필터링
    df = df[df['공급지역명'].isin(['서울', '경기', '인천'])]

    df_copy = pre_col_gen(df)

    # Kakao API로 위도와 경도 가져오기
    df_copy = process_address_data(df_copy)
    
    # 주소 분리 및 업데이트 
    df_copy = process_address(df_copy)

    # 위도와 경도가 None인 행만 선택하여 다시 처리
    df_none = df_copy[(df_copy['위도'].isnull()) & (df_copy['경도'].isnull())].copy()
    
    if not df_none.empty:
        df_none_processed = process_address_data(df_none)
        df_copy.loc[df_none.index, ['위도', '경도']] = df_none_processed[['위도', '경도']]
    
    df_copy.drop("정제_공급위치", axis=1, inplace=True)
    
    return df, df_copy


In [4]:
df = pd.read_csv("병합_청약매물_목록_정보.csv", encoding='cp949')
df, df_copy = lat_lon_pipeline(df)
df_merged = pd.merge(df, df_copy, on='공고번호', how='left')

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_87211/785735821.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("병합_청약매물_목록_정보.csv", encoding='cp949')


In [5]:
df_merged.to_csv('병합_청약매물_목록_정보_위도경도추가.csv', index=False, encoding='cp949')

In [3]:
df_merged = pd.read_csv("병합_청약매물_목록_정보_위도경도추가.csv", encoding='cp949')

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_91358/1948573066.py:1: DtypeWarning: Columns (36,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df_merged = pd.read_csv("병합_청약매물_목록_정보_위도경도추가.csv", encoding='cp949')


In [5]:
df_merged

,주택관리번호,공고번호,주택명,주택구분코드,주택구분코드명,주택상세구분코드,주택상세구분코드명,분양구분코드,분양구분코드명,공급지역코드,...,접수건수,경쟁률,최저당첨가점,최고당첨가점,평균당첨가점,위도,경도,시도,시군구,읍면동
0,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,0,(△5),NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
1,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,15,3.00,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
2,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,3,-,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
3,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,10,-,NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
4,2025000043,2025000043,부천 JY 포에시아,1,APT,1,민영,0,분양주택,410,...,1,(△4),NaN,NaN,NaN,37.485377,126.776922,경기도,부천시,원미구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21226,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21227,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21228,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
21229,2020000305,2020000305,수원세류2단지 2BL,1,APT,3,국민,1,분양전환 가능임대,410,...,0,NaN,NaN,NaN,NaN,37.263418,127.006154,경기도,수원시,권선구
